In [1]:
import pandas as pd
import requests
from functools import reduce

In [2]:
interest_periods = [
   ["2016-2021", "Soybeans", "China", ["IL", "IA", "MN", "IN", "NE"], "2018", 25],
   ["2016-2021", "Corn", "China", ["IA", "IL", "NE", "MN", "IN"], "2018", 25],
   ["2016-2021", "Cotton", "China", ["TX", "GA", "AR", "MS", "AL"], "2018", 25],
   ["2016-2021", "Wheat", "China", ["KS", "ND", "MT", "WA", "OK"], "2018", 25],
   ["2016-2021", "Rice", "China", ["AR", "CA", "LA", "MS", "MO"], "2018", 25],
   ["2014-2019", "Rice", "EU", ["AR", "CA", "LA", "MS", "MO"], "2015", 20],
   ["2007-2012", "Barley", "China", ["ID", "MT", "ND", "WY", "WA"], "2008", 25],
   ["2011-2016", "Cotton", "India", ["TX", "GA", "AR", "MS", "AL"], "2012", 30]
]

interest_periods_df = pd.DataFrame(interest_periods, columns=['Period', 'Crop', 'Trade Partner', "States", "Effect Year", "Tariff Rate"])

In [3]:
interest_periods_df

,Period,Crop,Trade Partner,States,Effect Year,Tariff Rate
0,2016-2021,Soybeans,China,"[IL, IA, MN, IN, NE]",2018,25
1,2016-2021,Corn,China,"[IA, IL, NE, MN, IN]",2018,25
2,2016-2021,Cotton,China,"[TX, GA, AR, MS, AL]",2018,25
3,2016-2021,Wheat,China,"[KS, ND, MT, WA, OK]",2018,25
4,2016-2021,Rice,China,"[AR, CA, LA, MS, MO]",2018,25
5,2014-2019,Rice,EU,"[AR, CA, LA, MS, MO]",2015,20
6,2007-2012,Barley,China,"[ID, MT, ND, WY, WA]",2008,25
7,2011-2016,Cotton,India,"[TX, GA, AR, MS, AL]",2012,30


# Pre-downloaded Data

In [4]:
interest_periods_df["Period"].unique()

array(['2016-2021', '2014-2019', '2007-2012', '2011-2016'], dtype=object)

In [5]:
interest_periods_df["Trade Partner"].unique()

array(['China', 'EU', 'India'], dtype=object)

Need FX data for USDCNY, USDMXN, USDEUR, USDINR 2000-Current

In [6]:
interest_periods_df["Crop"].unique()

array(['Soybeans', 'Corn', 'Cotton', 'Wheat', 'Rice', 'Barley'],
      dtype=object)

Need Commodity Price data for above crops 2000-Current

# USDA NASS

In [7]:
USDA_NASS_KEY = "BD016300-5F00-3A88-8B4B-C04608B1E1D6"
nass_base_url = f"https://quickstats.nass.usda.gov/api/api_GET/"

In [8]:
def nass_build_request(api_key, params, base_url):
    url = f"{base_url}?key={api_key}"
    for key, value in params.items():
        if isinstance(value, list):
            value = ",".join(map(str, value))
        url += f"&{key}={value.replace(' ', '+')}"
    return url

def nass_yield_json_to_df(data, state):
   df = pd.DataFrame(data)
   df = df[df['unit_desc'] == 'BU / ACRE']
   df['year'] = pd.to_numeric(df['year'])
   df = df[df['year'] <= int(end_year)]

   df = df[~df['Value'].str.contains(r'\(D\)', na=False)]
   df['Value'] = pd.to_numeric(df['Value'].str.replace(',', ''))
   
   df['reference_period'] = pd.to_datetime(df['year'].astype(str))
   df = df.groupby('year')['Value'].mean().reset_index()
   df['reference_period'] = pd.to_datetime(df['year'].astype(str))
   df = df[['reference_period', 'Value']]
   df.columns = ['Date', f'Production_{state}']
   return df

def nass_production_json_to_df(data, state):
   df = pd.DataFrame(data)
   df = df[df['unit_desc'] == 'BU'] 
   df['year'] = pd.to_numeric(df['year'])
   df = df[df['year'] <= int(end_year)]
   
   df = df[~df['Value'].str.contains(r'\(D\)', na=False)]
   df['Value'] = pd.to_numeric(df['Value'].str.replace(',', ''))
   
   df['reference_period'] = pd.to_datetime(df['year'].astype(str))
   df = df.groupby('year')['Value'].mean().reset_index()
   df['reference_period'] = pd.to_datetime(df['year'].astype(str))
   df = df[['reference_period', 'Value']]
   df.columns = ['Date', f'Production_{state}']
   return df


for row in interest_periods_df.values:
    
    all_yield_dfs = []
    all_production_dfs = []

    start_year, end_year = row[0].split("-")[0], row[0].split("-")[1]
    commodity = row[1].upper()
    state_alphas = row[3]

    for state in state_alphas:
        params = {
            "commodity_desc": commodity,
            "year__GE": start_year,
            "year__LE": end_year,
            "statisticcat_desc": "YIELD",
            "state_alpha": state
            }

        url = nass_build_request(USDA_NASS_KEY, params, nass_base_url)
        response = requests.get(url)
        
        state_df = nass_yield_json_to_df(response.json()['data'], state)
        all_yield_dfs.append(state_df)

        params = {
            "commodity_desc": commodity,
            "year__GE": start_year,
            "year__LE": end_year,
            "statisticcat_desc": "PRODUCTION",
            "state_alpha": state
            }
        
        url = nass_build_request(USDA_NASS_KEY, params, nass_base_url)
        response = requests.get(url)
        state_df = nass_production_json_to_df(response.json()['data'], state)
        all_production_dfs.append(state_df)
        
    
    yield_df = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), all_yield_dfs)
    production_df = reduce(lambda left, right: pd.merge(left, right, on='Date', how='outer'), all_production_dfs)

    final_df = pd.DataFrame({
    'Date': yield_df['Date'],
    'Average_Yield': yield_df.filter(like='Yield_').mean(axis=1),
    'Average_Production': production_df.filter(like='Production_').mean(axis=1)
    })
    final_df.to_csv(f"datasets/model1/USDA_NASS/{row[0]}-{commodity}.csv", index=False)

# USDA FAS

In [ ]:
USDA_FAS_KEY = "dNaDKzvoHmPLHDlEIbugYSWufZvp0fH6hZWe6GdD"
fas_base_url = f"https://api.fas.usda.gov/api/esr/commodities"

headers = {
   'accept': 'application/json',
   'X-Api-Key': USDA_FAS_KEY
}

In [10]:
commodity_codes = {
    "WHEAT" : [101,102,103,104,105,106,107,201],
    "BARLEY" : [301],
    "CORN": [401],
    "SOYBEANS"  : [801,901,902],
    "COTTON" : [1201,1202,1203,1301,1401,1402,1403,1404],
    "RICE" : [1498,1499,1501,1502,1504,1505]
}
country_codes = {
    "EU" : 1,
    "MEXICO" : 2010,
    "INDIA" : 5330,
    "CHINA" : 5700
}

In [12]:
def fas_trade_to_df(data):
   if not data or 'results' in data:
       return pd.DataFrame()
       
   df = pd.DataFrame(data)
   required_cols = ['weekEndingDate', 'weeklyExports', 'accumulatedExports', 'outstandingSales', 'grossNewSales']
   
   if not all(col in df.columns for col in required_cols):
       return pd.DataFrame()
   
   df['weekEndingDate'] = pd.to_datetime(df['weekEndingDate'])
   df['month'] = df['weekEndingDate'].dt.to_period('M')
   
   monthly_data = df.groupby('month').agg({
       'weeklyExports': 'sum',
       'accumulatedExports': 'last', 
       'outstandingSales': 'last',
       'grossNewSales': 'sum'
   }).reset_index()
   
   monthly_data['month'] = monthly_data['month'].astype(str)
   monthly_data['Date'] = pd.to_datetime(monthly_data['month'])
   return monthly_data[['Date', 'weeklyExports', 'accumulatedExports', 'outstandingSales', 'grossNewSales']]

In [14]:
for index, row in interest_periods_df.iterrows():
   start_year, end_year = row['Period'].split('-')
   commodity_codes_list = commodity_codes[row['Crop'].upper()]
   country_code = country_codes[row['Trade Partner'].upper()]
   
   all_data = []
   for year in range(int(start_year), int(end_year)+1):
       monthly_totals = None
       
       for commodity_code in commodity_codes_list:
           url = f"https://api.fas.usda.gov/api/esr/exports/commodityCode/{commodity_code}/countryCode/{country_code}/marketYear/{year}"
           response = requests.get(url, headers=headers)
           
           if response.status_code == 200 and response.json():
               df = fas_trade_to_df(response.json())
               if not df.empty:
                   if monthly_totals is None:
                       monthly_totals = df
                   else:
                       numeric_cols = ['weeklyExports', 'accumulatedExports', 'outstandingSales', 'grossNewSales']
                       monthly_totals[numeric_cols] += df[numeric_cols]
       
       if monthly_totals is not None:
           all_data.append(monthly_totals)
           
   if all_data:
       final_df = pd.concat(all_data).sort_values('Date')
       final_df['Commodity'] = row['Crop']
       final_df['Country'] = row['Trade Partner']
       final_df.to_csv(f"datasets/model1/USDA_FAS/{row['Period']}-{row['Crop'].upper()}.csv", index=False)

# USDA ARMS

In [15]:
USDA_ARMS_KEY = "FeoZKsLJQrekDhA6rMUukEbVdqGezgKfagzdXZCE"
arms_base_url = "https://api.ers.usda.gov/data/arms/surveydata"

In [18]:
def arms_build_request(api_key, params, arms_base_url):
   """
   Constructs the API request URL with the given parameters.
   """
   url = f"{arms_base_url}?api_key={api_key}"
   for key, value in params.items():
       if isinstance(value, list):
           value = ",".join(map(str, value))
       url += f"&{key}={value.replace(' ', '+')}"
   return url

def fetch_data(api_key, params):
   """
   Fetches data from the API using the constructed request.
   """
   url = arms_build_request(api_key, params, arms_base_url)
   print(f"Request URL: {url}")  # Debugging: Print the request URL


   response = requests.get(url)


   if response.status_code == 200:
       print("Request successful!")
       return response.json()
   else:
       print(f"Error: Received status code {response.status_code}")
       print(f"Response text: {response.text}")
       return None
   
def json_to_df(data):
   """
   Converts JSON data to a pandas DataFrame.
   """
   df = pd.DataFrame(data)
   print("Available columns:", df.columns)  # Debugging: Check available columns


   # Ensure required columns exist in the DataFrame
   if 'stat_Year' in df.columns and 'estimate' in df.columns:
       df['Year'] = df['stat_Year']
       df['Value'] = pd.to_numeric(df['estimate'], errors='coerce')
       return df
   else:
       print("Error: Required columns are missing from the data.")
       return None

In [26]:
params = {
       "year": list(range(2000,2024)),
       "state": "MN",
       "variable": "vprodtot,igcfi,incfi,infi,etot,evtot,accrop,ivalinv,icrop,frwcnum,tacres,efrent,evcwork,evfertc,evseedp,atot,dtot,frdanum,frcrnum",
       "statisticcat_desc": "ALL",
       "report": "Farm Business Income Statement"
   }
response_data = fetch_data(USDA_ARMS_KEY, params)
arms_data = json_to_df(response_data["data"])
if arms_data is not None:
    # Check for duplicates
    print("Number of duplicates:", arms_data.duplicated(subset=['Year', 'variable']).sum())
    # Handle duplicates by keeping the mean estimate per 'Year' and 'variable'
    arms_data = (
        arms_data.groupby(['Year', 'variable'], as_index=False)
        .agg({'Value': 'mean'})  # Aggregate duplicate entries
    )


    # Pivot the data
    arms_data = arms_data.pivot(index='Year', columns='variable', values='Value')

Request URL: https://api.ers.usda.gov/data/arms/surveydata?api_key=FeoZKsLJQrekDhA6rMUukEbVdqGezgKfagzdXZCE&year=2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023&state=MN&variable=vprodtot,igcfi,incfi,infi,etot,evtot,accrop,ivalinv,icrop,frwcnum,tacres,efrent,evcwork,evfertc,evseedp,atot,dtot,frdanum,frcrnum&statisticcat_desc=ALL&report=Farm+Business+Income+Statement
Request successful!
Available columns: Index(['stat_Year', 'stateName', 'reportName', 'farmType', 'category',
       'categoryValue', 'category2', 'category2Value', 'variableId',
       'variableName', 'variableSequence', 'variableLevel', 'variableGroup',
       'variableGroupId', 'variableUnit', 'variableDesc', 'variableIsInValid',
       'estimate', 'median', 'statistic', 'rse', 'unreliable_Est', 'dec_Disp',
       'series', 'serie_Dim', 'series2', 'serie2_Dim', 'series_Element',
       'serie_Element_Dim', 'series2_Element', 'serie2_Element_Dim', 'fi

In [28]:
arms_data.to_csv("datasets/model1/USDA_ARMS_DATAPOINTS.csv")

### FULL DATA DESCRIPTION

#### Market Pricing [Daily]
- BARLEY
- CORN
- COTTON
- RICE
- SOYBEANS
- WHEAT

#### FX Rates [Daily]
- USDCNY
- USDEUR
- USDINR
- USDMXN

#### USDA_FAS [Monthly]
- monthlyExports
- accumulatedExports
- outstandingSales
- grossNewSales

#### USDA_NASS [Annual]
- Average_Yield
- Average_Production

#### USDA_ARMS [Annual]
- Crop sales
- Fertilizer and chemicals
- Gross cash farm income
- Machine-hire and custom work
- Net cash farm income
- Net farm income
- Rent 

#### Other
- Agricultural Raw Material Price Index [Daily] IMF
- Drought Index [Tridaily] https://www.climate.gov/maps-data/data-snapshots/data-source/drought-monitor
- Fed Funds Interest Rate [Monthly] 
- Food Price Index [Daily] IMF
- IGC Grain and Oilseeds Index [Daily] https://www.igc.int/markets/marketinfo-goi.aspx